In [35]:
import math
from pprint import pprint

In [36]:
with open("input.txt", "rt") as f:
    instructions, _nodes = f.read().strip().split("\n\n")
    nodes = {}
    for node in _nodes.split("\n"):
        from_, to = node.split(" = ")
        to = to.strip("()").split(", ")
        nodes[from_] = to

Part 1

In [37]:
def infinite(iterator):
    while True:
        for e in iterator:
            yield e

In [42]:
current_position = "AAA"
for step, next_instruction in enumerate(infinite(instructions), start=1):
    if next_instruction == "R":
        current_position = nodes[current_position][1]
    elif next_instruction == "L":
        current_position = nodes[current_position][0]
    
    if current_position == "ZZZ":
        break

print(step)

14257


Part 2
  
I'm assuming that every path will create a loop eventually.  
It's possible that the path from start node to first z node  
isn't part of the loop, so I need to keep track of that separately:  
<code>
  ┌─────┐  
  │start├────┐  
  └─────┘    │  
         ┌───▼────────┐  
         │first z node│  
         └──▲───────┬─┘  
            └───────┘  
</code>
In order to detect a loop I can check if we came back to the first z node  
on the same instruction index, since it is possible that we visit the same  
z node twice:  
<code>
  ┌─────┐  
  │start├────┐  
  └─────┘    │  
         ┌───▼────────┐  ┌────────────┐  
         │first z node├──►first z node│  
         └───────▲────┘  └──┬─────────┘  
                 │          │  
                 └──────────┘  
</code>
It is possible that on one path there are multiple z nodes:  
<code>
  ┌─────┐  
  │start├────┐  
  └─────┘    │  
         ┌───▼────────┐  ┌────────────┐  
         │first z node├──►first z node│  
         └───────▲────┘  └──────┬─────┘  
                 │              │  
                 │          ┌───▼──┐  
                 └───...────┤z node│  
                            └──────┘  
</code>
I need to keep track of how many steps it takes to get to each z node from the  
start of the loop. With that I'll be able to quickly check if path is in z node  
for any given step.  


In [39]:
def infinite_enumerate(iterator):
    while True:
        for e in enumerate(iterator):
            yield e

In [40]:
a_nodes = set(from_ for from_ in nodes.keys() if from_.endswith("A"))
z_nodes = set(from_ for from_ in nodes.keys() if from_.endswith("Z"))

positions = [
    {
        "start": node,
        "current": node,
        "first_z": {
            "node": None,
            "instruction_index": None,
            "step": None,
        },
        "z_loop": [0],
    }
    for node in a_nodes
]
for position in positions:
    for step, (instruction_index, next_instruction) in enumerate(infinite_enumerate(instructions), start=1):  # fmt: skip
        
        if next_instruction == "R":
            position["current"] = nodes[position["current"]][1]
        elif next_instruction == "L":
            position["current"] = nodes[position["current"]][0]

        if position["current"].endswith("Z"):
            if position["first_z"]["node"] is None:
                position["first_z"]["node"] = position["current"]
                position["first_z"]["instruction_index"] = instruction_index
                position["first_z"]["step"] = step
            else:
                step -= position["first_z"]["step"]
                position["z_loop"].append(step)
                if instruction_index == position["first_z"]["instruction_index"] and position["first_z"]["node"] == position["current"]:
                    break

pprint(positions)

[{'current': 'DXZ',
  'first_z': {'instruction_index': 268, 'node': 'DXZ', 'step': 21251},
  'start': 'LTA',
  'z_loop': [0, 21251]},
 {'current': 'KHZ',
  'first_z': {'instruction_index': 268, 'node': 'KHZ', 'step': 15871},
  'start': 'TTA',
  'z_loop': [0, 15871]},
 {'current': 'HRZ',
  'first_z': {'instruction_index': 268, 'node': 'HRZ', 'step': 19099},
  'start': 'NJA',
  'z_loop': [0, 19099]},
 {'current': 'HSZ',
  'first_z': {'instruction_index': 268, 'node': 'HSZ', 'step': 12643},
  'start': 'BGA',
  'z_loop': [0, 12643]},
 {'current': 'ZZZ',
  'first_z': {'instruction_index': 268, 'node': 'ZZZ', 'step': 14257},
  'start': 'AAA',
  'z_loop': [0, 14257]},
 {'current': 'KRZ',
  'first_z': {'instruction_index': 268, 'node': 'KRZ', 'step': 19637},
  'start': 'KJA',
  'z_loop': [0, 19637]}]


Shortcut

Edge cases I was worried about aren't here, so I can check lowest common multiple and call it a day.

In [41]:
math.lcm(*(p["z_loop"][-1] for p in positions))

16187743689077

Bonus

How to check if we are in z node for any given step on the looped path.  
Tested on last example in the puzzle.

In [34]:
def is_in_z(position: dict, step: int) -> bool:
    if step < position["first_z"]["step"]:
        return False

    step -= position["first_z"]["step"]

    for z_step in position["z_loop"][1:]:
        if step % z_step == 0:
            return True
    return False


for s in range(1, 10):
    print(f"{s = }")
    for position in positions:
        print(f"{is_in_z(position, s) = }")
print()

s = 1
is_in_z(position, s) = False
is_in_z(position, s) = False
s = 2
is_in_z(position, s) = True
is_in_z(position, s) = False
s = 3
is_in_z(position, s) = False
is_in_z(position, s) = True
s = 4
is_in_z(position, s) = True
is_in_z(position, s) = False
s = 5
is_in_z(position, s) = False
is_in_z(position, s) = False
s = 6
is_in_z(position, s) = True
is_in_z(position, s) = True
s = 7
is_in_z(position, s) = False
is_in_z(position, s) = False
s = 8
is_in_z(position, s) = True
is_in_z(position, s) = False
s = 9
is_in_z(position, s) = False
is_in_z(position, s) = True

